In [5]:
%matplotlib inline
import pandas as pd
import urllib2
from urllib2 import urlopen
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import matplotlib.ticker as tick
import scipy.stats as sp
import statsmodels.api as sm
from pandas.stats.api import ols
from datetime import datetime
from bs4 import BeautifulSoup
from pylab import rcParams
import platform
rcParams['figure.figsize'] = 15, 10
import re
import os
import glob
import urllib
#import lxml
import HTMLParser
#from lxml import etree
from cStringIO import StringIO

In [6]:
def make_soup(url):
    # opens webpage for use in BeautifulSoup    
    html = urlopen(url).read()
    return BeautifulSoup(html, "lxml")

In [7]:
if platform.system() == 'Windows':
    drive = 'M:'
else:
    drive = '/media/p/Transcend/'
 
rootlocal = drive + '/PROJECTS/POWMOW/'
OUT = rootlocal + 'all.csv'

# Well Log Scraper

Scrapes well Logs from Water Rights website

In [8]:
# Water Rights win number to begin search
winbegin = 0
space = 100
path = 'C:\\PROJECTS\\WR_DATA\\V2\\'

winend = winbegin + space

In [ ]:
while winbegin < 60000:
         
    soup = []
    win = []
       
    # opens waterrights webpage by win   
    for i in range(winbegin,winend):
        try:
            soup.append(make_soup('http://waterrights.utah.gov/cgi-bin/docview.exe?Folder=welllog'+str(i)))
            win.append(str(i))
        except TypeError:        
            pass
    
        souplist = []
        win1 = []
        
    # finds printed well log in opened water rights webpage and adds them to the list souplist 
    for j in range(len(soup)):
        try:
            souplist.append(soup[j].find('a', href=re.compile('^http://waterrights.utah.gov/docSys/v907/.*'))['href'])
            win1.append(win[j])
        except TypeError:
            pass
    
    soupsite = []    
    souptext = []
    win  = []    
    
    for j in range(len(souplist)):
        try:
            soupsite.append(make_soup(souplist[j]))
            win.append(win1[j])
        except Warning:
            pass
    
    win1 = []    
    
    for j in range(len(souplist)):
        try:
            souptext.append(soupsite[j].get_text())
            win1.append(win[j])
        except HTMLParser.HTMLParseError:
            pass
    
    g=[]
    
    for j in range(len(souptext)):
        g.append(path + 'log' + str(win1[j]).zfill(5) + '.txt')    
        b = open(g[j], 'w')
        b.write(souptext[j])

    winbegin = winend
    winend = winbegin + space

# File Parser

This code searches through text captures of Water Rights html well files.

In [12]:
filepath = 'C:\\PROJECTS\\WR_DATA\\*.txt'

In [15]:
def raises(exception_types, func, *args, **kw):
    try:
        func(*args, **kw)
    except exception_types:
        return True
    else:
        return False

## Construction Parser

In [13]:
scrnpath = 'C:\\PROJECTS\\WR_DATA\\const\\'

In [18]:
for f in glob.glob(filepath): 
    win = []
    win.append(int(os.path.split(f)[1][3:8]))   
    text = open(f).read()    

    # grab section out of each text file for parsing
    welltest = []
    welltest.append(text[text.find('CASING:'):text.find('\r\n\r\n',text.find('CASING:'))])

    rev = []
    win1 = []

    for i in range(len(welltest)):
        if len(welltest[i]) > 10:    
            rev.append(str(re.sub('\r\n      +', '\n',welltest[i])))
            win1.append(str(win[i]))
        else:
            pass  

    # Make separators consistent so that parsing is easier
    for i in range(len(rev)):
        rev[i] = re.sub('\r\n','\n',rev[i])
        rev[i] = re.sub(',',';',rev[i])
        rev[i] = re.sub('  +',',',rev[i])
        rev[i] = re.sub('\n,','\n',rev[i])
        rev[i] = re.sub('\n\n','\n',rev[i])
        
    rv=[]
   
    # parse lines and columns
    for i in range(len(rev)):
        # line parsing
        rv.append(rev[i].split('\n')) 
        # column parsing
        #print rv[i]
        for j in range(3,len(rv[i])):
            t = rv[i][j].split(',')
            #print t
            
            if " " in t[0]:
                t[0] = re.sub(' ',',',t[0])
            elif "+" in t[0]:
                t[0] = re.sub('\+','-',t[0])
                
            if len(t) == 1:
                pass
            elif len(t) == 3:
                if raises(ValueError, float, t[2]):
                    rv[i][j] = win1[i] + ',' + t[0] + ',' + t[1] + ',' + t[2] + ',,'
                else:
                    rv[i][j] = win1[i] + ',' + t[0] + ',' + t[1] + ',,,' + t[2]
            elif len(t) == 4:
                if raises(ValueError, float, t[2]):
                    rv[i][j] = win1[i] + ',' + t[0] + ',' + t[1] + ',' + t[2] + ',,' + t[3]
                else:
                    rv[i][j] = win1[i] + ',' + t[0] + ',' + t[1] + ',,' + t[2] + ',' + t[3] 
            elif len(t)==5:
                rv[i][j] = win1[i] + ',' + t[0] + ',' + t[1] + ',' + t[2] + ',' + t[3] + ',' + t[4]

        rev[i] = '\n'.join(rv[i])

    g=[]
    
    for i in range(len(rev)):
        g.append(scrnpath + 'const' + str(win1[i]).zfill(5) + '.csv')    
        b = open(g[i], 'w')
        b.write(rev[i])

In [19]:
fout=open(scrnpath+"out.csv","a")

testcombpath = scrnpath+ "*.csv"

# Write Header
fout.write("WIN,From (ft),To (ft),Material,Gage (in),Diameter (in)")

for scrnfile in glob.glob(testcombpath):
    f = open(scrnfile)
    lines = f.readlines()[3:]
    
    fout.write("\n")    
    for g in lines:
        fout.write(g)    
    f.close() # not really needed
fout.close()

## Screen Parser

In [ ]:
scrnpath = 'C:\\PROJECTS\\WR_DATA\\screens\\'

for f in glob.glob(filepath): 
    win = []
    win.append(int(os.path.split(f)[1][3:8]))   
    text = open(f).read()    
    
    
# Well Tests ------------------------------------------------------------------------------

    welltest = []    
    welltest.append(text[text.find('SCREENS/PERFORATIONS:'):text.find('\r\n\r\n',text.find('SCREENS/PERFORATIONS:'))])
    

    rev = []
    win1 = []

    #print texty[0:3]
    for i in range(len(welltest)):
        if len(welltest[i]) > 10:    
            rev.append(str(re.sub('\r\n      +', '\n',welltest[i])))
            win1.append(str(win[i]))
        else:
            pass  

    for i in range(len(rev)):
        rev[i] = re.sub('\r\n','\n',rev[i])
        rev[i] = re.sub(',',';',rev[i])
        rev[i] = re.sub('  +',',',rev[i])
        rev[i] = re.sub('\n,','\n',rev[i])
        rev[i] = re.sub('\n\n','\n',rev[i])
        
    rv=[]
   
    for i in range(len(rev)):    
        rv.append(rev[i].split('\n'))
        for j in range(len(rv[i])):    
            if rv[i][j].count(',')==2:
                rv[i][j] = win1[i] + ',' + rv[i][j] + ', , , ' 
            elif rv[i][j].count(',')==3: 
                rv[i][j]= win1[i] + ',' + rv[i][j] +  ', , '
            elif rv[i][j].count(',')==1: 
                rv[i][j]= win1[i] + ',' + rv[i][j] +  ', , , , '
            elif rv[i][j].count(',')==4: 
                rv[i][j]= win1[i] + ',' + rv[i][j] +  ', '
            elif rv[i][j].count(',')==0:
                rv[i][j]= win1[i] + ',' + rv[i][j] +  ', , , , , '
            elif rv[i][j].count(',')==5: 
                rv[i][j]= win1[i] + ',' + rv[i][j]
        rev[i] = '\n'.join(rv[i])

    g=[]
    
    for i in range(len(rev)):
        g.append(scrnpath + 'scrn' + str(win1[i]).zfill(5) + '.csv')    
        b = open(g[i], 'w')
        b.write(rev[i])


In [ ]:
fout=open(scrnpath+"out.csv","a")
# first file:

testcombpath = scrnpath+"*.csv"
for line in open(scrnpath + "scrn00003.csv"):
    fout.write(line)
# now the rest:    
for scrnfile in glob.glob(testcombpath):
    f = open(scrnfile)
    lines = f.readlines()[3:]    
    fout.write("\n")    
    for g in lines:
        fout.write(g)    
    f.close() # not really needed
fout.close()

## Pumping Test Parser

In [ ]:
testpath = 'C:\\PROJECTS\\WR_DATA\\welltest\\'


for f in glob.glob(filepath): 
    win = []
    win.append(int(os.path.split(f)[1][3:8]))   
    text = open(f).read()    
    
    
# Well Tests ------------------------------------------------------------------------------

    welltest = []    
    welltest.append(text[text.find('WELL TESTS:'):text.find('\r\n\r\n\r\n ',text.find('WELL TESTS:'))])
    

    rev = []
    win1 = []

    #print texty[0:3]
    for i in range(len(welltest)):
        if len(welltest[i]) > 10:    
            rev.append(str(re.sub('\r\n       +', '\n',welltest[i])))
            win1.append(str(win[i]))
        else:
            pass  

    for i in range(len(rev)):
        rev[i] = re.sub('\r\n','\n',rev[i])
        rev[i] = re.sub(',',';',rev[i])
        rev[i] = re.sub('  +',',',rev[i])
        rev[i] = re.sub('\n,','\n',rev[i])
        rev[i] = re.sub('\n\n','\n',rev[i])
        
    rv=[]
   
    for i in range(len(rev)):    
        rv.append(rev[i].split('\n'))
        for j in range(len(rv[i])):    
            if rv[i][j].count(',')==2:
                rv[i][j] = win1[i] + ',' + rv[i][j] + ', , ' 
            elif rv[i][j].count(',')==3: 
                rv[i][j]= win1[i] + ',' + rv[i][j] +  ', '
            elif rv[i][j].count(',')==1: 
                rv[i][j]= win1[i] + ',' + rv[i][j] +  ', , , '
            elif rv[i][j].count(',')==4: 
                rv[i][j]= win1[i] + ',' + rv[i][j]    
            elif rv[i][j].count(',')==0:
                rv[i][j]= win1[i] + ',' + rv[i][j] +  ', , , , '
        rev[i] = '\n'.join(rv[i])

    g=[]
    
    for i in range(len(rev)):
        g.append(testpath + 'test' + str(win1[i]).zfill(5) + '.csv')    
        b = open(g[i], 'w')
        b.write(rev[i])



fout=open(testpath+"out.csv","a")
# first file:

testcombpath = testpath+"*.csv"
for line in open(testpath + "test00001.csv"):
    fout.write(line)
# now the rest:    
for testfile in glob.glob(testcombpath):
    f = open(testfile)
    lines = f.readlines()[3:]    
    fout.write("\n")    
    for g in lines:
        fout.write(g)    
    f.close() # not really needed
fout.close()

# System Scraper

In [ ]:
http://www.waterrights.utah.gov/cgi-bin/wuseview.exe?Modinfo=Indview&SYSTEM_ID=11247
http://www.waterrights.utah.gov/cgi-bin/wuseview.exe?Modinfo=Mgtview&SYSTEM_ID=11228

In [109]:
winbegin = 1000
space=1000
path = 'C:\\PROJECTS\\WR_DATA\\V3\\'

winend = winbegin + space

while winbegin < 4000:
         
    systemnm = []
       
    # opens waterrights webpage by win   
    for i in range(winbegin,winend):
        try:
            soup = make_soup('http://www.waterrights.utah.gov/cgi-bin/wuseview.exe?Modinfo=Pwsview&SYSTEM_ID='+str(i)).get_text()
            if "ERROR: Use UNITS undefined" in soup or len(soup) < 1000:
                pass
            else:
                systemnm.append(str(i))
                g = path + 'log' + str(i).zfill(6) + '.txt'
                b = open(g, 'w')
                b.write(soup.encode('utf-8').strip())   
                b.close()
        except TypeError:        
            pass
    
    winbegin = winend
    winend = winbegin + space
print("Done with lower limits")
winbegin = 10000
while winbegin < 14000:
         
    systemnm = []
       
    # opens waterrights webpage by win   
    for i in range(winbegin,winend):
        try:
            soup = make_soup('http://www.waterrights.utah.gov/cgi-bin/wuseview.exe?Modinfo=Pwsview&SYSTEM_ID='+str(i)).get_text()
            if "ERROR: Use UNITS undefined" in soup or len(soup) < 1000:
                pass
            else:
                systemnm.append(str(i))
                g = path + 'log' + str(i).zfill(6) + '.txt'
                b = open(g, 'w')
                b.write(soup.encode('utf-8').strip())   
                b.close()
        except TypeError:        
            pass
    
    winbegin = winend
    winend = winbegin + space
print("Done with upper limits")

In [110]:
winbegin = 1000
space=1000
path = 'C:\\PROJECTS\\WR_DATA\\V3\\'

winend = winbegin + space

while winbegin <= 5000:
         
    systemnm = []
       
    # opens waterrights webpage by win   
    for i in range(winbegin,winend):
        try:
            soup = make_soup('http://www.waterrights.utah.gov/cgi-bin/wuseview.exe?Modinfo=Indview&SYSTEM_ID='+str(i)).get_text()
            if "ERROR: Use UNITS undefined" in soup or len(soup) < 1000:
                pass
            else:
                systemnm.append(str(i))
                g = path + 'ind' + str(i).zfill(6) + '.txt'
                b = open(g, 'w')
                b.write(soup.encode('utf-8').strip())   
                b.close()
        except TypeError:        
            pass
    
    winbegin = winend
    winend = winbegin + space
    

winbegin = 10000

while winbegin <= 14000:
         
    systemnm = []
       
    # opens waterrights webpage by win   
    for i in range(winbegin,winend):
        try:
            soup = make_soup('http://www.waterrights.utah.gov/cgi-bin/wuseview.exe?Modinfo=Indview&SYSTEM_ID='+str(i)).get_text()
            if "ERROR: Use UNITS undefined" in soup or len(soup) < 1000: 
                pass
            else:
                systemnm.append(str(i))
                g = path + 'ind' + str(i).zfill(6) + '.txt'
                b = open(g, 'w')
                b.write(soup.encode('utf-8').strip())   
                b.close()
        except TypeError:        
            pass
    
    winbegin = winend
    winend = winbegin + space
    

In [111]:
winbegin = 0
space=1000
path = 'C:\\PROJECTS\\WR_DATA\\V2\\'

winend = winbegin + space

while winbegin <= 5000:
         
    systemnm = []
       
    # opens waterrights webpage by win   
    for i in range(winbegin,winend):
        try:
            soup = make_soup('http://www.waterrights.utah.gov/cgi-bin/wuseview.exe?Modinfo=Mgtview&SYSTEM_ID='+str(i)).get_text()
            if "ERROR: Use UNITS undefined" in soup or len(soup) < 1200:
                pass
            else:
                systemnm.append(str(i))
                g = path + 'Mgt' + str(i).zfill(6) + '.txt'
                b = open(g, 'w')
                b.write(soup.encode('utf-8').strip())   
                b.close()
        except TypeError:        
            pass
    
    winbegin = winend
    winend = winbegin + space
    
winbegin = 10000

while winbegin <= 15000:
         
    systemnm = []
       
    # opens waterrights webpage by win   
    for i in range(winbegin,winend):
        try:
            soup = make_soup('http://www.waterrights.utah.gov/cgi-bin/wuseview.exe?Modinfo=Mgtview&SYSTEM_ID='+str(i)).get_text()
            if "ERROR: Use UNITS undefined" in soup or len(soup) < 1200:
                pass
            else:
                systemnm.append(str(i))
                g = path + 'Mgt' + str(i).zfill(6) + '.txt'
                b = open(g, 'w')
                b.write(soup.encode('utf-8').strip())   
                b.close()
        except TypeError:        
            pass
    
    winbegin = winend
    winend = winbegin + space
    


# System Parser

In [181]:
scrnpath = 'C:\\PROJECTS\\WR_DATA\\V3\\*.txt'

indcode, systype, sysnum, link, sysname, city, county, syscat, huc, pwsid, deqcat = [],[],[],[],[],[],[],[],[],[],[]

files = glob.glob(scrnpath)

def gettext(strttext,endtext,text):
    b = text[text.find(strttext)+len(strttext):text.find(endtext,text.find(strttext))].strip()
    if len(b) < 1:
        return ''
    else:
        return b

g = {}
    
for f in range(len(files)): 
    text = open(files[f]).read()
    html = 'http://www.waterrights.utah.gov/cgi-bin/wuseview.exe?Modinfo=Pwsview&SYSTEM_ID='
    sysnum.append(int(os.path.split(files[f])[1][3:9]))
    linknum = int(os.path.split(files[f])[1][3:9])
    systype.append((os.path.split(files[f])[1][0:3]))
    link.append(html+str(linknum))
    
    sysname.append(gettext('System  Name:','Address:',text))
    city.append(gettext('City:','State:',text))
    county.append(gettext('County:','Primary Use:',text))
    syscat.append(gettext('Primary Use:','Standard',text))
    huc.append(gettext('Hydro Unit Code:','Public',text))
    pwsid.append(gettext('Public Water System ID:','DEQ',text))
    deqcat.append(gettext('DEQ System Category:','\n',text))
    indcode.append(gettext('Standard Industrial Code:','Dual',text))
    sid = gettext('Public Water System ID:','DEQ',text)

sysdict = {'systype':systype, 'systemnum': sysnum, 'link':link, 'sysname':sysname, 'city':city, 
           'county':county, 'syscat':syscat, 'indust code':indcode,
          'huc':huc, 'pwsid':pwsid,'deqcat':deqcat}

systems = pd.DataFrame(sysdict)

# System and Source Parser

In [41]:
scrnpath = 'C:\\PROJECTS\\WR_DATA\\V3\\*.txt'

indcode, systype, sysnum, link, sysname, city, county, syscat, huc, pwsid, deqcat, numberofsources = [],[],[],[],[],[],[],[],[],[],[],[]

files = glob.glob(scrnpath)

def gettext(strttext,endtext,snip):
    b = snip[snip.find(strttext)+len(strttext):snip.find(endtext,snip.find(strttext))].strip()
    if snip.find(strttext) == -1:
        return np.nan
    else:
        return b

g = {}

source, system, systemid, pls, sourcetype, sourceuse, win, wrnum, sourcecode = [],[],[],[],[],[],[],[],[]

for f in range(len(files)): 
    text = open(files[f]).read()
    systemname = gettext('System  Name:','Address:',text)
    sid = gettext('Public Water System ID:','DEQ',text)
    srcind = [m.start() for m in re.finditer('Source Summary', text)]
    
    html = 'http://www.waterrights.utah.gov/cgi-bin/wuseview.exe?Modinfo=Pwsview&SYSTEM_ID='
    sysnum.append(int(os.path.split(files[f])[1][3:9]))
    linknum = int(os.path.split(files[f])[1][3:9])
    systype.append((os.path.split(files[f])[1][0:3]))
    link.append(html+str(linknum))
    
    sysname.append(gettext('System  Name:','Address:',text))
    city.append(gettext('City:','State:',text))
    county.append(gettext('County:','Primary Use:',text))
    syscat.append(gettext('Primary Use:','Standard',text))
    huc.append(gettext('Hydro Unit Code:','Public',text))
    pwsid.append(gettext('Public Water System ID:','DEQ',text))
    deqcat.append(gettext('DEQ System Category:','\n',text))
    indcode.append(gettext('Standard Industrial Code:','Dual',text))
    sid = gettext('Public Water System ID:','DEQ',text)
    
    numberofsources.append(len(srcind))
    
    for i in range(len(srcind)):

        if i == len(srcind)-1:
            subtext = text[srcind[i]:-1]
        else:
            subtext = text[srcind[i]:srcind[i+1]]
        source.append(gettext('Source Name:','\n',subtext))
        pls.append(gettext('PLS Location:','\n',subtext))
        sourcetype.append(gettext('Source Type:','\n',subtext))
        sourceuse.append(gettext('Primary Use:','\n',subtext))
        win.append(gettext('Well ID Number:','(C',subtext))
        sourcecode.append(gettext('DEHN Source Code:','\n',subtext))
        wrnum.append(gettext('Water Right Numbers:','\n',subtext))
        system.append(systemname)
        systemid.append(sid)              


sysdict = {'systype':systype, 'systemnum': sysnum, 'link':link, 'sysname':sysname, 'city':city, 
           'county':county, 'syscat':syscat, 'indust code':indcode,
          'huc':huc, 'pwsid':pwsid,'deqcat':deqcat}

systems = pd.DataFrame(sysdict)
        
sourcedict = {'source':source, 'system id': systemid, 'system':system, 'pls':pls, 'source type': sourcetype,
          'source use':sourceuse, 'win':win, 'wrnum':wrnum, 'DEHN source id':sourcecode}

sources = pd.DataFrame(sourcedict)

In [78]:
scrnpath = 'C:\\PROJECTS\\WR_DATA\\V3\\*.txt'

indcode, systype, sysnum, link, sysname, city, county, syscat, huc, pwsid, deqcat = [],[],[],[],[],[],[],[],[],[],[]

files = glob.glob(scrnpath)


g = {}
    
for f in range(len(files)): 
    text = open(files[f]).read()
    st = ' Annual Use Info (Acft) \r\n'
    strt = text.find(st)

    fin = text.find('\r\n \r',strt)
    pwsid = gettext('Public Water System ID:','DEQ',text)
    if strt == -1:
        pass
    else:
        table = text[strt:fin]
        table = re.sub('\r\n +','\r\n',table)
        table = re.sub(' +',',',table)
        table = re.sub('\r\n','\n',table)
        g[pwsid] = pd.read_csv(StringIO(table),skiprows=1)
        #g[f]['system']=int(text[text.find("SYSTEM_ID=")+10:])
systemuse = pd.concat(g)

In [79]:
systemuse

Commercial  Domestic Industrial  Institutnl  Other   Stock Tota  \
      0         0.00     47.25          0        0.00      0    0.00  NaN   
01001 0       218.76    398.28      32.64        4.00      0    0.00  NaN   
      1       172.66    366.51      34.94       56.73      0  317.02  NaN   
      2       283.78    347.79       0.07       52.30      0   21.94  NaN   
      3         0.00      0.00          0        0.00      0    0.00  NaN   
      4         0.00      0.00          0        0.00      0    0.00  NaN   
      5         0.00      0.00          0        0.00      0    0.00  NaN   
      6       182.58   1460.66      36.52       36.52      0  109.55  NaN   
      7       130.18   1041.44      26.04       26.04      0   78.11  NaN   
      8       132.42   1059.39      26.48       26.48      0   79.45  NaN   
      9        61.94    495.55      12.39       12.39      0   37.17  NaN   
      10      109.72    877.77      10.97       43.89      0   54.86  NaN   
      11      171.99    917.28      11.47       11.47      0   34.40  NaN   
      12        0.00      0.00          0        0.00      0    0.00  NaN   
      13      207.41    220.08      57.06        2.48      0   14.09  NaN   
      14      207.41    220.08      57.06        2.48      0   14.09  NaN   
      15        0.00      0.00          0        0.00      0    0.00  NaN   
      16      107.50    860.02       21.5       35.83  53.75    0.00  NaN   
      17      142.45    712.25          0       94.97      0    0.00  NaN   
      18        0.00      0.00          0        0.00      0    0.00  NaN   
      19        0.00      0.00          0        0.00      0    0.00  NaN   
      20      249.98    370.54      15.04        0.00  12.22    0.00  NaN   
      21      294.25    393.89     188.31        0.00  13.51    0.00  NaN   
      22       40.34    328.34      123.3       37.65      0    0.00  NaN   
      23       60.50    476.11     140.75       86.65      0    0.00  NaN   
      24       49.15    580.88     123.96       63.53      0    0.00  NaN   
      25       43.91    531.85      98.67      632.14      0    0.00  NaN   
      26       31.98    491.57       91.9      578.95      0    0.00  NaN   
      27      144.25    570.92       79.8      365.22      0    0.00  NaN   
      28      125.83    328.39      61.38        0.00      0    0.00  NaN   
...              ...       ...        ...         ...    ...     ...  ...   
29108 4         0.00      0.00          0        0.00      0    0.00  NaN   
      5         0.00      6.54          0        0.00      0    0.00  NaN   
      6         0.00      1.04          0        0.00      0    0.00  NaN   
      7         0.00      1.04          0        0.00      0    0.00  NaN   
      8         0.00      0.60          0        0.00      0    0.00  NaN   
      9         0.00      0.60          0        0.00      0    0.00  NaN   
      10        0.00      0.60          0        0.00      0    0.00  NaN   
      11        0.00      0.60          0        0.00      0    0.00  NaN   
      12        0.00      0.60          0        0.00      0    0.00  NaN   
      13        0.00      0.60          0        0.00      0    0.00  NaN   
      14        0.00      0.60          0        0.00      0    0.00  NaN   
      15        0.00      0.23          0        0.00      0    0.00  NaN   
      16        0.00      0.23          0        0.00      0    0.00  NaN   
      17        0.00      0.24          0        0.00      0    0.00  NaN   
      18        0.00      0.23          0        0.00      0    0.00  NaN   
      19        0.00      0.21          0        0.00      0    0.00  NaN   
      20        0.00      0.20          0        0.00      0    0.00  NaN   
      21        0.00      0.00          0        0.00      0    0.00  NaN   
      22        0.00      0.17          0        0.00      0    0.00  NaN   
      23        0.00      0.16          0        0.00      0    0.00  NaN   
      

In [46]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('C:\\PROJECTS\\WR_DATA\\' + 'systems_and_sources.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
systems.to_excel(writer, sheet_name='systems')
sources.to_excel(writer, sheet_name='sources')
systemuse.to_excel(writer, sheet_name='system_use')
# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [ ]:
scrnpath = 'C:\\PROJECTS\\WR_DATA\\V3\\*.txt'

indcode, systype, sysnum, link, sysname, city, county, syscat, huc, pwsid, deqcat = [],[],[],[],[],[],[],[],[],[],[]

files = glob.glob(scrnpath)

def gettext(strttext,endtext,text):
    b = text[text.find(strttext)+len(strttext):text.find(endtext,text.find(strttext))].strip()
    if len(b) < 1:
        return ''
    else:
        return b

g = {}
    
for f in range(len(files)): 
    text = open(files[f]).read()
    st = 'Year     Domestic Commercial'
    strt = text.find(st)
    fin = text.find('\r\n \r\n',strt)
    pwsid = gettext('Public Water System ID:','DEQ',text)
    if strt == -1:
        pass
    else:
        table = text[strt:fin]
        table = re.sub('\r\n +','\r\n',table)
        table = re.sub(' +',',',table)
        table = re.sub('\r\n','\n',table)
        g[pwsid] = pd.read_csv(StringIO(table))
        #g[f]['system']=int(text[text.find("SYSTEM_ID=")+10:])
        